In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 24 11:09:17 2024
@author: suraj2.desai
"""
from sshtunnel import SSHTunnelForwarder
import mysql.connector
import psycopg2
from datetime import timedelta
from datetime import datetime
import pandas as pd
import time
import json
import numpy as np
import pymysql
# Define your SSH and MySQL details
ssh_host = '10.134.72.5'
ssh_port = 22
ssh_user = 'fynd'
ssh_private_key =  r'/Users/Nitin14.Patil/Downloads/ssh_key.fynd'
 

mysql_host = '172.17.0.38'  # This should be 127.0.0.1 because we are using SSH tunneling
mysql_port = 3306
mysql_user = 'fynd_avis_read'
mysql_password = 'fynd_avis_read!2022'
mysql_database = 'avis'
fromDate = datetime(2024, 4, 1, 0, 0, 0)
toDate = datetime(2024, 9, 21, 15, 56, 0) 
from_timestamp = int(fromDate.timestamp())
to_timestamp = int(toDate.timestamp())


query = """
SELECT s.fynd_order_id,
s.id as "shipment_id",
FROM_UNIXTIME(o.created_ts) as "order_date",
pm.mode,
ss.status,
b.seller_identifier as "article_code",
b.id as "bag_id",
i.name as "item_name",
i.brand,
i.l1_category ,
i.l2_category ,
i.l3_category ,
b.item_id,
b.gstin_code,
b.quantity ,
b.journey_type,
s2.code as "store_code",
b.store_id ,
s.store_invoice_id ,
s.credit_note_id,
payment_mode_id ,
source,ordering_channel,
s.delivery_awb_number,
s.delivery_address_json,
b.prices,
b.article_json,
b.meta,
b.applied_promos,
s.meta as "shipment_meta"
FROM avis.`order` o
LEFT JOIN shipment s ON o.fynd_order_id = s.fynd_order_id
LEFT JOIN (SELECT shipment_id, status, created_at
FROM (SELECT shipment_id,status,created_at, ROW_NUMBER() OVER (PARTITION BY shipment_id ORDER BY created_ts DESC ,updated_ts DESC, id DESC) AS row_num FROM shipment_status) AS subquery
WHERE row_num = 1) ss ON s.id = ss.shipment_id
LEFT JOIN bag b on s.id = b.shipment_id
LEFT JOIN item i on b.item_id = i.id
LEFT JOIN payment_mode pm on o.payment_mode_id = pm.id
LEFT JOIN store s2 on b.store_id = s2.id
WHERE o.fynd_order_id like 'SS%' 
 
and
o.created_ts >= {from_timestamp} 
AND o.created_ts <= {to_timestamp}
""".format(from_timestamp=from_timestamp, to_timestamp=to_timestamp)
with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_user,
    ssh_pkey=ssh_private_key,
    remote_bind_address=(mysql_host, mysql_port)
) as tunnel:
    # Connect to the MySQL database using pymysql
    connection = pymysql.connect(
        user=mysql_user,
        password=mysql_password,
        host='127.0.0.1',
        port=tunnel.local_bind_port,
        database=mysql_database
    )
    # Create a cursor object
    cursor = connection.cursor()
    # Disable ONLY_FULL_GROUP_BY for the current session
    cursor.execute("SET sql_mode=(SELECT REPLACE(@@sql_mode,'ONLY_FULL_GROUP_BY',''));")
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    # Convert the result to a Pandas DataFrame
    columns = [column[0] for column in cursor.description]
    df = pd.DataFrame(result, columns=columns)
 
    df1 = df[['fynd_order_id','shipment_id','article_code','bag_id','delivery_address_json','prices','article_json','meta','shipment_meta']]
    df2 = df1.values.tolist()
    df3 = df2[1][6]
    result = pd.DataFrame()
    result1 = pd.DataFrame()
    result2 = pd.DataFrame()
    result3 = pd.DataFrame()
    result4 = pd.DataFrame()
    df5 = pd.DataFrame()
    df8 = pd.DataFrame()
    for n in range(len(df2)):
        df3 = df2[n][6]
        if df3 is not None:
            df4 = json.loads(df3)
            df5 = pd.json_normalize(df4)[['size', 'price_marked']]
            df5['fynd_order_id'] = df2[n][0]
            df5['shipment_id'] = df2[n][1]
            df5['article_code'] = df2[n][2]
            df5['bag_id'] = df2[n][3]
            df5['identifier'] = [df4['identifier']]
            df5 = df5[['fynd_order_id','shipment_id','article_code','bag_id','size','price_marked','identifier']]
        if df3 is None:
            df5['fynd_order_id'] = df2[n][0]
            df5['shipment_id'] = df2[n][1]
            df5['article_code'] = df2[n][2]
            df5['bag_id'] = df2[n][3]
            df5['size'] = ''
            df5['price_marked'] = ''
            df5['identifier'] = ''
            df5 = df5[['fynd_order_id','shipment_id','article_code','bag_id','size','price_marked','identifier']]
        result = pd.concat([result, df5], axis=0, ignore_index=True)
        #df6 = pd.json_normalize(df4, meta=['size', 'price_marked', 'identifier.ean','identifier.sku_code'])
 
 
    for n in range(len(df2)):
        df3 = df2[n][4]
        df4 = json.loads(df3)
        df5 = pd.json_normalize(df4)
        df5['fynd_order_id'] = df2[n][0]
        df5['shipment_id'] = df2[n][1]
        df5['article_code'] = df2[n][2]
        df5['bag_id'] = df2[n][3]
        result1 = pd.concat([result1, df5], axis=0, ignore_index=True)
 
    for n in range(len(df2)):
        df3 = df2[n][5]
        if df3 is not None:
            df4 = json.loads(df3)
            df5 = pd.json_normalize(df4['amount_paid']['amount'])
            df6 = pd.json_normalize(df4['price_marked']['amount'])
            df7 = pd.json_normalize(df4['discount']['amount'])
            df14 =pd.json_normalize(df4['coupon_effective_discount']['amount'])
            df15 =pd.json_normalize(df4['promotion_effective_discount']['amount'])                      
            df8 = pd.concat([df5, df6, df7,df14,df15], axis=1, ignore_index=True)
            df8 = df8[[0,2,4,6,8]]
            df8 = df8.rename(columns={0: 'amount_paid', 2: 'price_marked', 4: 'discount',6:'coupon_applied',8:'promo_applied'})
            df8['fynd_order_id'] = df2[n][0]
            df8['shipment_id'] = df2[n][1]
            df8['article_code'] = df2[n][2]
            df8['bag_id'] = df2[n][3]
            df8 = df8[['fynd_order_id','shipment_id','article_code','bag_id','amount_paid','price_marked','discount','coupon_applied','promo_applied']]
        if df3 is None:
            df8['fynd_order_id'] = df2[n][0]
            df8['shipment_id'] = df2[n][1]
            df8['article_code'] = df2[n][2]
            df8['bag_id'] = df2[n][3]
            df8['amount_paid'] = ''
            df8['price_marked'] = ''
            df8['discount'] = ''
            df8['coupon_applied'] = ''
            df8['promo_applied'] = ''
            df8 = df8[['fynd_order_id','shipment_id','article_code','bag_id','amount_paid','price_marked','discount','coupon_applied','promo_applied']]
        result2 = pd.concat([result2, df8], axis=0, ignore_index=True)
    for n in range(len(df2)):
        df3 = df2[n][7]
        if df3 is not None:
            df4 = json.loads(df3)
            if 'serial_numbers' in df4:
                df5['serial_numbers'] = [df4['serial_numbers']]
                df5['fynd_order_id'] = df2[n][0]
                df5['shipment_id'] = df2[n][1]
                df5['article_code'] = df2[n][2]
                df5['bag_id'] = df2[n][3]
            if 'serial_numbers' not in df4:
                df5['serial_numbers'] = ''
                df5['fynd_order_id'] = df2[n][0]
                df5['shipment_id'] = df2[n][1]
                df5['article_code'] = df2[n][2]
                df5['bag_id'] = df2[n][3]
        if df3 is None:
            df5['serial_numbers'] = ''
            df5['fynd_order_id'] = df2[n][0]
            df5['shipment_id'] = df2[n][1]
            df5['article_code'] = df2[n][2]
            df5['bag_id'] = df2[n][3]
        result3 = pd.concat([result3, df5], axis=0, ignore_index=True)
    for n in range(len(df2)):
        df3 = df2[n][8]
        df4 = json.loads(df3)
        if any(key.startswith('article_level_discount') for key in df4):
                for key in df4:
                    if key.startswith('article_level_discount'):
                        df5 = pd.json_normalize(df4[key])
                        df5['fynd_order_id'] = df2[n][0]
                        df5['bag_id'] = df2[n][3]
        else:
           df5['bagId'] = ''
           df5['lineNo'] = ''
           df5['articleId'] = ''
           df5['articlePrice'] = ''
           df5['discountAmount'] = ''
           df5['fynd_order_id'] = df2[n][0]
           df5['bag_id'] = df2[n][3]
        result4 = pd.concat([result4, df5], axis=0, ignore_index=True)
 
 
    final_dataframe = pd.merge(df, result, on = ['fynd_order_id','shipment_id','article_code','bag_id'], how = 'inner')
    final_dataframe1 = pd.merge(final_dataframe, result1, on = ['fynd_order_id','shipment_id','article_code','bag_id'], how = 'inner')
    final_dataframe2 = pd.merge(final_dataframe1, result2, on = ['fynd_order_id','shipment_id','article_code','bag_id'], how = 'inner')
    final_dataframe3 = pd.merge(final_dataframe2, result3, on = ['fynd_order_id','shipment_id','article_code','bag_id'], how = 'inner')
    final_dataframe4 = pd.merge(final_dataframe3, result4, on = ['fynd_order_id','bag_id'], how = 'left')
    
 
final_dataframe4['Final_status']  = np.where(final_dataframe4['credit_note_id'].isna(), final_dataframe4['status'], 'handed_over_to_customer') 


# Sample dataframe


# Create column 'c' where column 'b' contains 'xx'
final_dataframe4['SMKITTI'] = ''
final_dataframe4.loc[final_dataframe4['applied_promos'].str.contains('SMKITTI',na=False), 'SMKITTI'] = 'Yes'

print(df)

# and
# ss.status not in ('pending','payment_failed')




date = time.strftime("%Y%m%d%H%M%S")
final_dataframe4 = final_dataframe4.replace('\n', '-', regex=True)
final_dataframe4.to_csv(rf'\Users\Nitin14.Patil\Downloads\order_discount{date}.txt', sep = '|')

In [6]:
import psycopg2
import time
import datetime
from dateutil import parser
from datetime import timedelta
import pandas as pd
import numpy as np
import json
from sshtunnel import SSHTunnelForwarder
import psycopg2
import paramiko
# SSH server information
ssh_host = '10.134.72.5'
ssh_port = 22
ssh_username = 'fynd'
private_key_path = '/Users/Nitin14.Patil/Downloads/ssh_key.fynd'
# Database server information
db_host = '172.17.0.16'
db_port = 5432
db_username = 'fynd_hogwarts_read'
db_password = 'fynd_hogwarts_read!2022'
 
"""
sql = '''select
t.primary_identifier,
t.state,
 
 
t.status,
 
 
t.data,
 
 
te.payload
 
 
from task t
 
 
join
 
 
task_events te
 
 
on t.uid = te.task_id
 
 
where
 
t.state='placed' and
 
t.status not in ('stopped', 'failed') and
 
identifier_type = 'fynd_order' and
 
te.payload <> 'None' and
 
t.created_on >= '{compare_date}' '''.format(compare_date = datetime.datetime.now() - datetime.timedelta(minutes= 10))
 
"""
"""
sql = '''select
 
t.primary_identifier,
 
 
t.state,
 
 
t.status,
 
 
t.data,
 
 
te.payload
 
 
from task t
 
 
join
 
 
task_events te
 
 
on t.uid = te.task_id
 
 
where
 
t.state='placed' and
 
t.status not in ('stopped', 'failed') and
 
t.identifier_type = 'fynd_order' and
 
t.primary_identifier in (

'FY6568B8A80E1D112AD7',
'FY656B303C0E2FC06551',
'FY656F36720EB3099EA5',
'FY6572BE090EA692F0BE',
'FY6578640B0E70227AE9',
'FY657AB1CB0ECAC47D9F',
'FY6568A93A0EBE573EC8',
'FY656C88140EC03B0AE6',
'FY656DB6A10EC71E0A3D',
'FY65770E5A0ECA1990DD',
'FY657348D60E9B5168B4',
'FY6583E5110E4E0C2C8C',
'FY65689DDA0E6CC3505B',
'FY657441830E3CB1F028',
'FY6579A5A40ED2C94D33',
'FY6579BD6F0E7CB90A8B',
'FY656B4A790EF348557B',
'FY656F0A950EB9FF651C',
'FY65E079140EF9162D52'
                         
                         
) and
 
te.payload <> 'None'
 
'''


"""
sql = '''select 
 
 
t.primary_identifier,
 
 
t.state,
 
 
t.status,
 
 
t.data,
 
 
te.payload
 
 
from task t
 
 
join
 
 
task_events te
 
 
on t.uid = te.task_id
 
 
where
 
t.state='placed' and
 
t.status not in ('stopped', 'failed') and
 
identifier_type = 'fynd_order' and
 
te.payload <> 'None' and
 
t.created_on >= '{fromDate}' and
 
t.created_on <= '{toDate}' '''.format(fromDate = datetime.datetime(2024, 9, 15, 0, 0, 0, 0), toDate = datetime.datetime(2024, 9, 27, 0, 0, 0, 0))


ssh_key = paramiko.RSAKey.from_private_key_file(private_key_path)
 
 
 
with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_pkey=ssh_key,
    remote_bind_address=(db_host, db_port)
) as tunnel:

    conn = psycopg2.connect(
        host='localhost',
        port=tunnel.local_bind_port,
        user=db_username,
        password=db_password,
        database='hogwarts'
    )
 
 
    cursor = conn.cursor()
    cursor.execute("select version()")    
    data = cursor.fetchone()
    print("Connection established to: ",data)
 
 
    cursor.execute(sql)
    results = cursor.fetchall()
    print(sql)
 
    cursor.close()
    conn.close()
data=results


list_json=[]
vas_json=[]
for t in results:
    list_json.append(list(t))
for n in range(len(list_json)):   
    a=list_json[n][4]
    a=eval(a)
    if 'products' in a and any(product.get('ITEM_VAL') == 'VAS' for product in a['products']):
    
        vas_json.append(a)

meta_d=pd.DataFrame()
for n1 in range(len(vas_json)):
    keys_to_extract = ['temp_order_id', 'customer_name','contact_number','billing_address','email_id','order_created_date']
    extracted_values = [[d[key] for key in keys_to_extract] for d in vas_json]
    
    df = pd.DataFrame(extracted_values, columns=keys_to_extract)

    ajc=pd.json_normalize(vas_json[n1])
    Article_aj=pd.json_normalize(ajc['products'])
    meta=Article_aj[0].to_dict()
    meta_df=pd.json_normalize(meta)
    
    meta_df1=meta_df[['0.REF_NO','0.article_id']]
    
    meta_d=pd.concat([meta_d,meta_df1])




meta_d=meta_d.reset_index()
meta_d=meta_d.drop(columns=['index'])


try:

    for n2 in range (len(df['billing_address'])):
    
    
        my_dict=df['billing_address'][n2]
    
        keys_to_get = ['flat_no', 'building_name','street','sector','city','state','pin']
    
        selected_values = {key: my_dict[key] for key in keys_to_get if key in my_dict}
    
    
    
    
        values = [str(value) for value in selected_values.values()]
    
        comma_separated_values = ' '.join(values)
    
    
    
        df['billing_address'][n2]=comma_separated_values
        final_df=pd.concat([df,meta_d],axis=1,ignore_index=True)
        column_names = ['temp_order_id', 'customer_name','contact_number','billing_address','email_id','order_created_date','imei_no','article_code']
        final_df.columns=(column_names)

        print(final_df)
        final_df.to_csv('VAS_data6.csv',index=False)
        
except:
    print('Messege:- There is no VAS entry Yesterday')

Connection established to:  ('PostgreSQL 14.12 on x86_64-pc-linux-gnu, compiled by Debian clang version 12.0.1, 64-bit',)
select 
 
 
t.primary_identifier,
 
 
t.state,
 
 
t.status,
 
 
t.data,
 
 
te.payload
 
 
from task t
 
 
join
 
 
task_events te
 
 
on t.uid = te.task_id
 
 
where
 
t.state='placed' and
 
t.status not in ('stopped', 'failed') and
 
identifier_type = 'fynd_order' and
 
te.payload <> 'None' and
 
t.created_on >= '2024-09-15 00:00:00' and
 
t.created_on <= '2024-09-27 00:00:00' 
            temp_order_id       customer_name contact_number  \
0    FY66E666920E272F5975           Md Zunaid     8674828030   
1    FY66E7C3CB0EF0C3DC17       Gaurav Gaurav     7015872176   
2    FY66E7C5D60E5460E819        Ayush Sharma     9622280930   
3    FY66E7D3620E65F4F01E          Salim Khan     8890602537   
4    FY66E7E35A0E6ADC3FC6         Ravi sorout     9991131384   
..                    ...                 ...            ...   
126  FY66E696870EDA5C5645  Mohd rashid Chiche 

            temp_order_id       customer_name contact_number  \
0    FY66E666920E272F5975           Md Zunaid     8674828030   
1    FY66E7C3CB0EF0C3DC17       Gaurav Gaurav     7015872176   
2    FY66E7C5D60E5460E819        Ayush Sharma     9622280930   
3    FY66E7D3620E65F4F01E          Salim Khan     8890602537   
4    FY66E7E35A0E6ADC3FC6         Ravi sorout     9991131384   
..                    ...                 ...            ...   
126  FY66E696870EDA5C5645  Mohd rashid Chiche     9622669000   
127  FY66E6A0210ECC25CBC4         Sunil Kumar     7830135551   
128  FY66E6C2D30EC7157411         Sintu Kumar     8941883976   
129  FY66E6CE370E00FFA2A9         Farmid Alam     6391242898   
130  FY66E6D8B90E1EFDAFF7      Vaibhav Saxena     9758569316   

                                       billing_address  \
0    000 Belaganj Bajidpur belaganj Chooti mahzid G...   
1    1xx Village siwana maal Siwana mal Near shiv m...   
2    2xx Ward no 2 Lucky bihar bishnah Bishnah Jamm...   

In [1]:
pwd

'/Users/nitin14.patil/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/Documents/jio_dev'

In [2]:
import os
import pandas as pd

# Folder path containing the files
folder_path = '/Users/nitin14.patil/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/Documents/python_work/PNL Finance/vas/'

# Get a list of all CSV files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through all the files and read them
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)  # Adjust if using other formats like pd.read_excel()
    dfs.append(df)

# Concatenate all the DataFrames along axis=0
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

# Display the resulting DataFrame
print(merged_df)

merged_df.to_csv('vas data.csv')

             temp_order_id             customer_name  contact_number  \
0     FY66B707FF0E27AD87A4             Anurag Sharma      9166836437   
1     FY66B709CD0E388EBF75           Sandeep Sandeep      9991565291   
2     FY66B70CFA0E87891159  Sujeet kumar Vishwakarma      7870878282   
3     FY66B714E20ED05A6D19             pradeep meher      9090800706   
4     FY66B715DC0E5086855D             Rishab Narang      9034122667   
...                    ...                       ...             ...   
9528  FY6650D7350ED3062098                    Pankaj      7053500492   
9529  FY6650D8E70E4BFE0CA3                      Arti      8810400051   
9530  FY6650DBCE0E3A5552F1              Bhavna Patel      9998631445   
9531  FY6650DAA40EAE58AD35             Sandeep kumar      9910967669   
9532  FY6650DB950E558F51B8              reetu rajput      9111627060   

                                        billing_address  \
0     Jaipur Jaipur Jaipur Jaipur Jaipur Rajasthan 3...   
1     540 ASANDA 